In [1]:
using HTTP, JSON, PrettyTables, JLD, MySQL, DotEnv, DataFrames, LinearAlgebra
include("functions/type_allocation_flexible.jl")
cfg = DotEnv.config("../.env")
#if you want to refresh the adjustment information set refresh to true
# to load saved data instead (which means you'll just be reading previously saved 
# adjustments, set refresh to false
refresh = true
files_path = cfg["files_path"]

"/home/peters/code/mapinator/estimation/current_estimates_and_files/"

The point of this worksheet is to adjust the size of the placements adjacency matrix
to accommodate the fact that the highest tiers are oversampled.  Eventually this should not be needed,
but at this point it is the only way to produce a reasonable set of estimates.

This first part of the file is just reading in classification information.

In [2]:
#load adjacency matrix and custom row titles from create classification
placement_rates = load(files_path*"placement_rates.jld")["placement_rates"]
row_names = load(files_path*"row_names.jld")["names"]
#recreate constants from create classification in order to build new latex table
numtotal = size(placement_rates)[1]
NUM_TYPES = size(placement_rates)[2]
NUM_SINKS = numtotal - NUM_TYPES
if numtotal !== length(row_names) 
    throw(DomainError(length(row_names), "row_names doesn't match dimensions of the placements matrix"))
else
printstyled("names and placement matrix dimensions match\n"; color = :green)
end


names and placement matrix dimensions match


This next part of the file is an attempt to compute the corrections.  

The first part attempts to find the coverage by tiers using information from econjobmarket 
registrations data.

Set refresh equal to false at the top of the worksheet if you just want to see the current corrections.

In [3]:
if refresh
    query = """
    select t.type,count(*) as count from ftrecruiter.applicants a join ftrecruiter.organizations o 
    join ftrecruiter.institutions i join t_distribution t on a.degreeinst_oid=o.oid and 
    o.institution_id=i.institution_id and t.institution_id=i.institution_id 
    where a.degreetype=? and a.degreeendyear>=? and a.degreeendyear<=? 
    and t.algorithm_run_id=? group by type
    """
    params = ["Phd", "2017", "2023", 5]
    registrations = SBM_flexible.db_query(query, params)
    regs = registrations[:,:count]
    query = """
    select t.type,count(*) as count from ftrecruiter.applicants a join ftrecruiter.organizations o 
    join ftrecruiter.institutions i join t_distribution t on a.degreeinst_oid=o.oid 
    and i.institution_id=o.institution_id and t.institution_id=i.institution_id 
    where year(enrolldate)>=? and year(enrolldate)<=? and degreetype=? 
    and t.algorithm_run_id=? and aid not in (select aid from applicant_results) group by t.type
    """
    params = ["2018", "2023", "Phd", 5]
    ms = SBM_flexible.db_query(query, params)
    miss = ms[:, :count]
    proportion_missing = ms[!,"count"]./registrations[!, "count"]
    s = []
    for x in proportion_missing
        push!(s,1/(1-x))
    end
    data = hcat(regs, miss, s)
    save(files_path*"adjustment_matrix.jld", "data", data)
else
    data = load(files_path*"adjustment_matrix.jld")["data"]
    s = data[:,3]
    adjusted_placement_rates = round.(Int,placement_rates*Diagonal(I*s))
   
end

In [4]:
registrations

Row,type,count
,Int32?,Int64
1,1,2664
2,2,3635
3,3,4481
4,4,2284
5,5,1348


In [5]:
data

5×3 Matrix{Any}:
 2664   348  1.15026
 3635   825  1.29359
 4481  1750  1.64079
 2284  1169  2.04843
 1348   744  2.23179

In [6]:
#head = ["Tier 1", "Tier 2", "Tier 3", "Tier 4", "Tier 5"]
head = []
for i in 1:NUM_TYPES
    push!(head, string("Type ", i))
end
rowh = ["Registrations - 2018-23", "Registrants without placements", "Adjustment"]

pretty_table(transpose(data), header= head, row_labels=rowh, backend = Val(:text))

┌────────────────────────────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│                                │  Type 1 │  Type 2 │  Type 3 │  Type 4 │  Type 5 │
├────────────────────────────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│        Registrations - 2018-23 │    2664 │    3635 │    4481 │    2284 │    1348 │
│ Registrants without placements │     348 │     825 │    1750 │    1169 │     744 │
│                     Adjustment │ 1.15026 │ 1.29359 │ 1.64079 │ 2.04843 │ 2.23179 │
└────────────────────────────────┴─────────┴─────────┴─────────┴─────────┴─────────┘


In [7]:
if refresh
    open(files_path*"adjustment_table.tex", "w") do f
    SBM_flexible.pretty_table(
        f,
        data',
        header = head,
        row_labels = rowh,
        backend = Val(:latex)
        )
    end
end


In [8]:
Diagonal(I*s)
adjusted_placement_rates = round.(Int,placement_rates*Diagonal(I*s))

12×5 Matrix{Int64}:
 1417   476   299   100    0
 1139  1270   530   250   18
 1232  1863  1682   320   74
  323   733   758   965   96
    9    61    84    90  362
  664   775   481   143   45
  910   594   415   158   62
  100   206   235   133   65
   40    94   125   102  121
  470   965  1559  1188  984
  244   233   161    72   20
  269   485   697   367  156

In [9]:
if refresh
    save(files_path*"adjusted_placement_rates.jld", "adjusted_placement_rates", adjusted_placement_rates)
end  

In [10]:
SBM_flexible.nice_adjacency_table(adjusted_placement_rates,row_names)

┌───────────────────────────────────┬────────┬────────┬────────┬────────┬────────┬────────────┐
│                                   │ Tier 1 │ Tier 2 │ Tier 3 │ Tier 4 │ Tier 5 │ Row Totals │
├───────────────────────────────────┼────────┼────────┼────────┼────────┼────────┼────────────┤
│                 TYPE 1 (20 insts) │   1417 │    476 │    299 │    100 │      0 │       2292 │
│                 TYPE 2 (62 insts) │   1139 │   1270 │    530 │    250 │     18 │       3207 │
│                TYPE 3 (179 insts) │   1232 │   1863 │   1682 │    320 │     74 │       5171 │
│                TYPE 4 (352 insts) │    323 │    733 │    758 │    965 │     96 │       2875 │
│                TYPE 5 (568 insts) │      9 │     61 │     84 │     90 │    362 │        606 │
│         Public Sector (159 insts) │    664 │    775 │    481 │    143 │     45 │       2108 │
│        Private Sector (255 insts) │    910 │    594 │    415 │    158 │     62 │       2139 │
│              Postdocs (635 insts) │   

In [11]:
if refresh
    SBM_flexible.nice_adjacency_table(adjusted_placement_rates, row_names, files_path*"adjusted_adjacency_table.tex")
end